In [ ]:
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf


#티스토리 업로드시 최적화 코드
#from IPython.core.display import display, HTML
#display(HTML("<style>.container {width:90% !important;}</style>"))

# 2023.04.17
# TRAINING
Dataset Info.

train.csv [파일]

    ID : 샘플 고유 ID
    User-ID : 유저 고유 ID
    Book-ID : 도서 고유 ID
    유저 정보
    Age : 나이
    Location : 지역
    도서 정보
    Book-Title : 도서 명
    Book-Author : 도서 저자
    Year-Of-Publication : 도서 출판 년도 (-1일 경우 결측 혹은 알 수 없음)
    Publisher : 출판사
    Book-Rating : 유저가 도서에 부여한 평점 (0점 ~ 10점)


test.csv [파일]

    ID : 샘플 고유 ID
    User-ID : 유저 고유 ID
    Book-ID : 도서 고유 ID
    유저 정보
    Age : 나이
    Location : 지역
    도서 정보
    Book-Title : 도서 명
    Book-Author : 도서 저자
    Year-Of-Publication : 도서 출판 년도 (-1일 경우 결측 혹은 알 수 없음)
    Publisher : 출판사


sample_submission.csv [제출양식]

    ID : 샘플 고유 ID
    Book-Rating : 예측한 유저가 도서에 부여할 평점

In [1]:
import pandas as pd

In [2]:
import numpy as np
import random
import os
import gc

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [4]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv( 'test.csv')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

In [6]:
train.head(10)

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books
2,TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books
3,TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,1991.0,Avon
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book
5,TRAIN_000005,USER_00000,BOOK_206799,5,23.0,"sackville, new brunswick, canada",Neuromancer (Remembering Tomorrow),William Gibson,1995.0,Ace Books
6,TRAIN_000006,USER_00000,BOOK_239414,9,23.0,"sackville, new brunswick, canada",The Little Prince,Antoine de Saint-ExupÃ©ry,1982.0,Harvest Books
7,TRAIN_000007,USER_00000,BOOK_269070,0,23.0,"sackville, new brunswick, canada",Forests of the Heart (Newford),Charles de Lint,2001.0,Tor Books
8,TRAIN_000008,USER_00001,BOOK_190739,7,35.0,"wake forest, north carolina, usa",Self Matters : Creating Your Life from the Ins...,Phillip C. McGraw,2001.0,Free Press
9,TRAIN_000009,USER_00001,BOOK_210079,0,35.0,"wake forest, north carolina, usa",The Self Matters Companion : Helping You Creat...,Phillip C. McGraw,2002.0,Free Press


In [7]:
train[['User-ID','Location']].drop_duplicates().groupby(['User-ID'])['Location'].count().sort_values()

User-ID
USER_00000    1
USER_61443    1
USER_61442    1
USER_61441    1
USER_61440    1
             ..
USER_30605    1
USER_30604    1
USER_30603    1
USER_30622    1
USER_92100    1
Name: Location, Length: 83256, dtype: int64

User-ID를 기준으로 Location은 고유의 값이라는 것을 확인 할 수 있습니다. 즉, 평가를 해주는 유저의 지역은 고유의 값입니다.

In [7]:
train['City'] = [(i.split(',')[0]).lstrip().title() for i in train['Location']]
train['State'] = [(i.split(',')[1]).lstrip().title() for i in train['Location']]
train['Country'] = [(i.split(',')[2]).lstrip().title() for i in train['Location']]

In [8]:
test['City'] = [(i.split(',')[0]).lstrip().title() for i in test['Location']]
test['State'] = [(i.split(',')[1]).lstrip().title() for i in test['Location']]
test['Country'] = [(i.split(',')[2]).lstrip().title() for i in test['Location']]

In [12]:
train['City'].unique()

array(['Sackville', 'Wake Forest', 'Minneapolis', ..., 'Hampton Cove',
       'Vancouver/ Richmond', 'Castiglion Fiorentino'], dtype=object)

In [13]:
train['State'].unique()

array(['New Brunswick', 'North Carolina', 'Minnesota', ..., 'Shetland',
       'Wanganui', 'Vaucluse'], dtype=object)

In [14]:
train['Country'].unique()

array(['Canada', 'Usa', 'Germany', 'Spain', 'Australia', 'United Kingdom',
       '', 'Italy', 'Netherlands', 'Portugal', 'Sweden', 'Austria',
       'France', 'Finland', 'New Zealand', 'Singapore', 'Switzerland',
       'Philippines', 'New York', 'España', 'Malaysia', 'Belgium',
       'Japan', 'Denmark', 'Belize', 'England', 'Poland', 'Ireland',
       'Haiti', 'Trinidad And Tobago', 'Lk Ammerland', 'Tanzania',
       'Moldova', 'North Carolina', 'Bulgaria', 'Luxembourg',
       'South Africa', 'Czech Republic', 'Tennessee', 'Israel', 'Malta',
       'Petrolwar Nation', 'Kuwait', 'Peru', 'Ecuador', 'South Korea',
       'Virginia', 'Turkey', 'Nigeria', 'British Columbia', 'Brazil',
       'Missouri', 'Hungary', 'Indonesia', 'Kazakhstan', 'India', 'China',
       'Egypt', 'Croatia', 'Taiwan', 'Argentina', 'California',
       'Queensland', 'Ontario', 'Chile', 'Deutschland', 'Venezuela',
       'Galiza', 'Iceland', 'Illinois', 'Scotland', 'New Jersey',
       'Bolivia', 'Lithuania', 'F

Location을 ','로 나누면 첫 단어가 도시, 두번째가 State(주), 마지막 단어가 나라입니다. 해당 열을 새로 만들어줍니다.

In [11]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
# BERTopic 속도 향상을 위한 작업
!pip install --extra-index-url https://download.pytorch.org/whl/cu117 --extra-index-url https://pypi.nvidia.com git+https://github.com/facebookresearch/dinov2
# 토픽 모델링을 위한 버토픽(BERTopic) 다운
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117, https://pypi.nvidia.com
  Cloning https://github.com/facebookresearch/dinov2 to /tmp/pip-req-build-9e6s639x
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/dinov2 /tmp/pip-req-build-9e6s639x
  Resolved https://github.com/facebookresearch/dinov2 to commit f8969297dbe53373b4041bf47d997a8dcc8d2077
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 9.5 MB/

In [9]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
title = train['Book-Title'].drop_duplicates().to_list()

In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install stop-words

In [11]:
from stop_words import get_stop_words
# 불용어 리스트
languages = [
   'Arabic',
   'Bulgarian',
   'Catalan',
   'Czech',
   'Danish',
   'Dutch',
   'English',
   'Finnish',
   'French',
   'German',
   'Hungarian',
   'Indonesian',
   'Italian',
   'Norwegian',
   'Polish',
   'Portuguese',
   'Romanian',
   'Russian',
   'Spanish',
   'Swedish',
   'Turkish',
   'Ukrainian'
]
my_stop_word_list = []
for i in languages:
    my_stop_word_list.append(get_stop_words(i.lower()))
my_stop_word_list = sum(my_stop_word_list, [])
my_stop_word_list = my_stop_word_list + ['paperbook', 'hardcover']

In [12]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()



In [13]:
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from cuml.preprocessing import normalize

np.random.seed(113)
# 벡터화 모델
vectorizer_model = CountVectorizer(stop_words=my_stop_word_list, ngram_range=(1, 2), min_df=10)

# Create instances of GPU-accelerated UMAP and HDBSCAN (GPU사용 모델)
umap_model = UMAP(n_components=5, n_neighbors=300, min_dist=0.0, metric='cosine', random_state =113)
hdbscan_model = HDBSCAN(metric='euclidean', min_samples=10, gen_min_span_tree=True, prediction_data=True)
# 불용어 처리
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
# Create your representation model
representation_model = KeyBERTInspired(random_state = 113)

model = BERTopic(vectorizer_model=vectorizer_model,
                 ctfidf_model=ctfidf_model,
                 umap_model=umap_model, 
                 hdbscan_model=hdbscan_model,
                 representation_model=representation_model,
                 # 다국어를 위해 해당 임베딩 모델 사용
                 embedding_model="paraphrase-multilingual-MiniLM-L12-v2",
                 nr_topics="auto",
                 low_memory = True,
                 language="multilingual",
                 verbose = True,
                 top_n_words = 12,
                 min_topic_size = 500
                 )

model.fit(title)
topics, probs = model.transform(title)

Batches:   0%|          | 0/6808 [00:00<?, ?it/s]

2023-04-24 10:39:38,114 - BERTopic - Transformed documents to Embeddings
2023-04-24 10:40:07,323 - BERTopic - Reduced dimensionality
2023-04-24 10:40:56,015 - BERTopic - Clustered reduced embeddings
2023-04-24 10:41:11,800 - BERTopic - Reduced number of topics from 826 to 703


Batches:   0%|          | 0/6808 [00:00<?, ?it/s]

2023-04-24 10:42:20,240 - BERTopic - Reduced dimensionality
2023-04-24 10:42:20,862 - BERTopic - Predicted clusters


In [14]:
model.get_topic_info()

,Topic,Count,Name
0,-1,143885,-1_cuentos_leben_book series_novels paperback
1,0,4102,0_married_marriage_wife_bride
2,1,3386,1_cookbook_recipes_food_diet
3,2,2426,2_parenting_families_parent guide_mothers
4,3,2168,3_bible_jesus_christian_teachings
...,...,...,...
698,697,5,697_henry___
699,698,5,698_saint_bible_book_stories
700,699,5,699_celebration_celebrate_life_
701,700,5,700_tales_white_spell_


In [15]:
model.save("my_topics_model_usingGPU")

In [16]:
title = pd.DataFrame(title, columns = ['Book-Title'])

In [17]:
topic = pd.Series(topics, name = 'Topic_num')
topic

0         -1
1         56
2          7
3         -1
4         -1
          ..
217824    -1
217825    -1
217826    -1
217827    -1
217828    -1
Name: Topic_num, Length: 217829, dtype: int64

In [18]:
title = pd.concat([title, topic], axis =1)
title

,Book-Title,Topic_num
0,Road Taken,-1
1,Macbeth (New Penguin Shakespeare),56
2,Waverley (Penguin English Library),7
3,Mother Earth Father Sky,-1
4,She Who Remembers,-1
...,...,...
217824,Be Victorious (Be),-1
217825,Prodigals and Those Who Love Them: Study Guide,-1
217826,A Transatlantic Love Affair: Letters to Nelson...,-1
217827,He's My Brother: Former Racial Foes Offer Stra...,-1


In [19]:
title_t = test['Book-Title'].drop_duplicates().to_list()

In [20]:
topics_t, probs_t = model.transform(title_t)

Batches:   0%|          | 0/1857 [00:00<?, ?it/s]

2023-04-24 10:42:45,670 - BERTopic - Reduced dimensionality
2023-04-24 10:42:45,843 - BERTopic - Predicted clusters


In [21]:
topic_t = pd.Series(topics_t, name = 'Topic_num')
title_t = pd.DataFrame(title_t, columns = ['Book-Title'])
title_t = pd.concat([title_t, topic_t], axis =1)
title_t

,Book-Title,Topic_num
0,Birds of Prey: A Novel of Suspense,-1
1,Midnight Voices,4
2,Breaking Free : A Prescription for Personal an...,-1
3,Bitter Harvest,-1
4,Embraced by the Light,4
...,...,...
59403,Changelings (Castlemere Mysteries (Paperback)),-1
59404,MARKETING HIGH TECHNOLOGY,-1
59405,Sierra: Notes and Images from the Range of Light,-1
59406,From 60 Yards In : How to Master Golf's Short ...,140


In [22]:
# 나이 그룹화
labels = ['0-3','3-6','6-8','8-12','12-18','18-25','25-34','35-44','45-54','55-64','65-74','75+']
bins = [0, 3, 6, 8, 12, 18, 25, 34, 44, 54, 64, 74, 250]
train['Age_gb'] = pd.cut(train.Age, bins, labels = labels,include_lowest = True)

In [23]:
test['Age_gb'] = pd.cut(test.Age, bins, labels = labels,include_lowest = True)

In [24]:
from sklearn.preprocessing import OrdinalEncoder

FEATURE = ['User-ID', 'Book-Title','Book-Author','Publisher', 'City','State','Country', 'Topic_num','Age_gb']

train_lb = train.__deepcopy__() 
test_lb = test.__deepcopy__()
train_lb = pd.merge(train_lb, title, left_on = 'Book-Title', right_on = 'Book-Title', how = 'left')
test_lb = pd.merge(test_lb, title_t, left_on = 'Book-Title', right_on = 'Book-Title', how = 'left')

train_lb = train_lb.drop(columns = ['Book-ID', 'Location'])
test_lb = test_lb.drop(columns = ['Book-ID', 'Location'])

train_lb[FEATURE] = train_lb[FEATURE].astype(str) 
test_lb[FEATURE] = test_lb[FEATURE].astype(str)

for i in FEATURE:
    # train에는 없고, test에는 있는 원소는 -2 처리
    oe = OrdinalEncoder(handle_unknown='use_encoded_value',
                         unknown_value=-2)
    oe=oe.fit(train_lb[i].to_numpy().reshape(-1, 1))
    train_lb[i] = oe.transform(train_lb[i].to_numpy().reshape(-1, 1))
    test_lb[i] = oe.transform(test_lb[i].to_numpy().reshape(-1, 1))

In [25]:
X_train = train_lb.drop(columns = ['ID','Book-Rating'])
y_train = train_lb['Book-Rating']
x_test = test_lb.drop(columns = ['ID'])

In [29]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install optuna

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 5.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 7.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 kB 5.0 MB/s eta 0:00:0000:0100:01


In [26]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# random sampler
sampler = TPESampler(seed=113)

# define function
def objective(trial):


    cbrm_param = {
        'l2_leaf_reg':trial.suggest_float("l2_leaf_reg",2,20),
        'max_bin': trial.suggest_int("max_bin", 200, 400),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.01),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15]),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        'iterations':trial.suggest_int("iterations", 1000, 25000),
        #'random_strength' :trial.suggest_int('random_strength', 0, 10),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        #'min_child_samples': trial.suggest_int("min_child_samples", 5, 100),
        'eval_metric' : 'RMSE',
        'task_type': 'GPU',
        'random_state': 113,
        'use_best_model': True,
        'bootstrap_type' : 'Poisson',
        'early_stopping_rounds' : 100
    }

    FEATURE = ['User-ID', 'Book-Title','Book-Author','Publisher', 'City','State','Country', 'Topic_num','Age_gb']
    train_pool = Pool(data=X_train.astype('int'), label=y_train, cat_features=FEATURE)
    valid_pool = Pool(data=X_val.astype('int'), label=y_val, cat_features=FEATURE)

    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param)
    model_cbrm = model_cbrm.fit(train_pool, eval_set=valid_pool, 
                           verbose=0, early_stopping_rounds=100)
                           
    MSE = mean_squared_error(y_val, model_cbrm.predict(X_val.astype('int')))
    RMSE = np.sqrt(MSE)
    return RMSE
optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=1000, timeout = 28800)

[I 2023-04-24 10:42:50,455] A new study created in memory with name: no-name-1577ff10-4aaf-49c2-8fdf-1e6ec975d846
[I 2023-04-24 10:55:13,302] Trial 0 finished with value: 3.2561020301556667 and parameters: {'l2_leaf_reg': 17.335738869141586, 'max_bin': 214, 'learning_rate': 0.007851122663573896, 'max_depth': 11, 'min_data_in_leaf': 206, 'iterations': 17816, 'subsample': 0.8678165576938894}. Best is trial 0 with value: 3.2561020301556667.
[I 2023-04-24 11:28:38,044] Trial 1 finished with value: 3.261615088457823 and parameters: {'l2_leaf_reg': 13.569488628767122, 'max_bin': 393, 'learning_rate': 0.0012837039728039537, 'max_depth': 11, 'min_data_in_leaf': 30, 'iterations': 23053, 'subsample': 0.7999950950580053}. Best is trial 0 with value: 3.2561020301556667.
[I 2023-04-24 11:39:52,881] Trial 2 finished with value: 3.268007798830466 and parameters: {'l2_leaf_reg': 5.14603061225714, 'max_bin': 243, 'learning_rate': 0.0029368061475182937, 'max_depth': 7, 'min_data_in_leaf': 78, 'iteration

In [27]:
cbrm_trial = optuna_cbrm.best_trial
cbrm_trial_params = cbrm_trial.params
print('Best Trial: score {},\nparams {}'.format(cbrm_trial.value, cbrm_trial_params))

cbrm_trial_params

Best Trial: score 3.2538045234128337,
params {'l2_leaf_reg': 17.178091468605494, 'max_bin': 348, 'learning_rate': 0.007058948631189331, 'max_depth': 11, 'min_data_in_leaf': 118, 'iterations': 24635, 'subsample': 0.9984770090737676}


{'l2_leaf_reg': 17.178091468605494,
 'max_bin': 348,
 'learning_rate': 0.007058948631189331,
 'max_depth': 11,
 'min_data_in_leaf': 118,
 'iterations': 24635,
 'subsample': 0.9984770090737676}

In [30]:

from catboost import CatBoostRegressor 

model = CatBoostRegressor(random_seed = 113,
                          l2_leaf_reg = 17.178091468605494,
                          max_bin = 348,
                          learning_rate = 0.007058948631189331,
                          max_depth = 11,
                          min_data_in_leaf = 118,
                          iterations = 24635,
                          subsample = 0.9984770090737676,
                          eval_metric = 'RMSE',
                          task_type='GPU',
                          bootstrap_type = 'Poisson',
                          #early_stopping_rounds = 100,
                          verbose=500
                          )

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=113)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from catboost import CatBoostRegressor, Pool

y_valid_pred = 0*y_train
y_test_pred = 0

FEATURE = ['User-ID', 'Book-Title','Book-Author','Publisher', 'City','State','Country', 'Topic_num','Age_gb']

for i, (train_index, test_index) in enumerate(skfold.split(X_train, y_train)):    
    X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    train_pool = Pool(data=X_train_fold.astype('int'), label=y_train_fold, cat_features=FEATURE)
    valid_pool = Pool(data=X_valid_fold.astype('int'), label=y_valid_fold, cat_features=FEATURE)

    print( "\nFold ", i)
    
    fit_model = model.fit(train_pool, 
                          eval_set=valid_pool,
                          use_best_model=True
                          )
    print( "  N trees = ", model.tree_count_ )
        
    # Generate validation predictions for this fold
    pred = fit_model.predict(X_valid_fold.astype('int'))
    y_valid_pred.iloc[test_index] = pred
    
    # Predict value Clipping
    y_test_pred +=  np.clip(fit_model.predict(x_test[X_valid_fold.columns].astype('int')),0.0,10.0)
    
y_test_pred /= 5  # Average test set predictions



Fold  0


0:	learn: 3.8416705	test: 3.8413203	best: 3.8413203 (0)	total: 72.1ms	remaining: 29m 36s
500:	learn: 3.3605630	test: 3.3164502	best: 3.3164502 (500)	total: 40.3s	remaining: 32m 23s
1000:	learn: 3.3263330	test: 3.2944109	best: 3.2944109 (1000)	total: 1m 22s	remaining: 32m 32s
1500:	learn: 3.3050473	test: 3.2864588	best: 3.2864588 (1500)	total: 2m 5s	remaining: 32m 10s
2000:	learn: 3.2896029	test: 3.2825895	best: 3.2825895 (2000)	total: 2m 48s	remaining: 31m 41s
2500:	learn: 3.2755291	test: 3.2797053	best: 3.2797053 (2500)	total: 3m 31s	remaining: 31m 8s
3000:	learn: 3.2636802	test: 3.2778578	best: 3.2778578 (3000)	total: 4m 14s	remaining: 30m 33s
3500:	learn: 3.2528791	test: 3.2765587	best: 3.2765587 (3500)	total: 4m 57s	remaining: 29m 54s
4000:	learn: 3.2422623	test: 3.2754208	best: 3.2754193 (3998)	total: 5m 40s	remaining: 29m 16s
4500:	learn: 3.2320921	test: 3.2744606	best: 3.2744596 (4499)	total: 6m 23s	remaining: 28m 37s
5000:	learn: 3.2223078	test: 3.2736591	best: 3.2736587 (4999)

In [ ]:
sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['Book-Rating'] = y_test_pred
sub

In [ ]:
import seaborn as sns
sns.histplot(sub['Book-Rating'])

In [ ]:
sub.to_csv('submit_cat+bertopic_optuna3.csv', index=False,encoding="utf-8-sig")